In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Step 1: Dataset Directory
DATASET_PATH = "/content/drive/MyDrive/wastes/wastes"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [3]:
# Step 2: Image Preprocessing and Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [4]:
train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/wastes/wastes/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 725 images belonging to 9 classes.


In [5]:
val_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/wastes/wastes/test",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 90 images belonging to 9 classes.


In [6]:
# Step 3: Load Pretrained MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [7]:
# Step 4: Build Model
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

In [8]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
# Step 5: Train Model
EPOCHS = 20
history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 117s 5s/step - accuracy: 0.1830 - loss: 3.4497 - val_accuracy: 0.4444 - val_loss: 1.6476
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 143s 5s/step - accuracy: 0.4472 - loss: 1.5408 - val_accuracy: 0.5889 - val_loss: 1.3017
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 139s 4s/step - accuracy: 0.5567 - loss: 1.2854 - val_accuracy: 0.5444 - val_loss: 1.3618
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - accuracy: 0.6601 - loss: 1.0279 - val_accuracy: 0.5889 - val_loss: 1.4171
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 108s 5s/step - accuracy: 0.6602 - loss: 0.9065 - val_accuracy: 0.6222 - val_loss: 1.2441
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 97s 4s/step - accuracy: 0.6731 - loss: 0.9504 - val_accuracy: 0.5556 - val_loss: 1.6171
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 108s 5s/step - accuracy: 0.7196 - loss: 0.7729 - val_accuracy: 0.6111 - val_loss: 1.4657
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 115s 5s/step - accuracy: 0.7611 - loss: 0.6490 - val_accuracy: 0.5889 - va

In [10]:
# Step 6: Save Model
model.save("waste_classifier.h5")


In [11]:
# Step 7: Convert Model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Saved artifact at '/tmp/tmplinmw0ea'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 9), dtype=tf.float32, name=None)
Captures:
  134647031021008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134647028319056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134647028318480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134647031021200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134647031020624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134647031016592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134647028319248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134647028318288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134647028320208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134647028319824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13464702832059

In [12]:
# Save the TensorFlow Lite model
with open("waste_classifier.tflite", "wb") as f:
    f.write(tflite_model)

In [13]:
##testing the model
import tensorflow as tf
import numpy as np
import cv2
import os

# Load the trained model
model = tf.keras.models.load_model("waste_classifier.h5")

# Load the class labels (must match the dataset folder structure)
class_labels = ['biodegradable', 'plastic', 'metal', 'glass', 'e-waste']  # Modify based on your dataset

# Function to preprocess an image
def preprocess_image(image_path):
    img = cv2.imread(image_path)  # Read image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (224, 224))  # Resize to match model input size
    img = img / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Expand dimensions to match model input
    return img

# Test on a new image
image_path = "/content/drive/MyDrive/wastes/wastes/test/glass waste/glass waste (26).jpg"  # Replace with your image path
input_image = preprocess_image(image_path)

# Make prediction
prediction = model.predict(input_image)
predicted_class = np.argmax(prediction, axis=1)[0]  # Get the class index

# Print the predicted waste category
print(f"Predicted Waste Category: {class_labels[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Waste Category: glass
